# How I got the sex and age data
I pulled the data for sex and age using the cohort broswer gui and then run the table exporter app to save the data in CSV format.
This is not the programmtic way but for the few number of variables (here just two) it is far easier and straightforward and faster to do but for datasets with more number of variable, we should use jupyterlab with pysparck cluser and get the data programmtically.

Field ID for age at recruitment: 31
Filed ID for sex: 22001

- So, first I pulled this data and save it as CSV
- It lacks the FID, I would merge FID from the KING.kin0 output (Only 1 FID for each IID, I checked for this)
- I also check for how many common IIDs are in the KING and the agesex data
- Then I create a agesex file in this format: https://snipar.readthedocs.io/en/latest/input%20files.html#agesex-file
- Then king file and agesex data is ready for doing ibd inference with Snipar

In [1]:
import pandas as pd

In [3]:
fp = "project-J1VzYJQJQp6z67g2f0Vvb2kK:/user_data__Mahdi/i70_g6_CSF/c_med/array_data__kinship_degree1.kin0"

In [6]:
%%bash
dx download "project-J1VzYJQJQp6z67g2f0Vvb2kK:/user_data__Mahdi/i70_g6_CSF/c_med/array_data__kinship_degree1.kin0"
ls

array_data__kinship_degree1.kin0


In [8]:
df_king = pd.read_csv("array_data__kinship_degree1.kin0", sep = '\t')
df_king.head()

,FID1,ID1,FID2,ID2,N_SNP,HetHet,IBS0,HetConc,HomIBS0,Kinship,IBD1Seg,IBD2Seg,PropIBD,InfType
0,-1,-1,4696286,4696286,755432,0.0965,0.0064,0.3985,0.1260,0.2464,0.5543,0.2121,0.4893,FS
1,-11,-11,4734664,4734664,725130,0.0837,0.0000,0.3301,0.0005,0.2461,0.9995,0.0000,0.4997,PO
2,-12,-12,4734664,4734664,724414,0.0829,0.0000,0.3280,0.0006,0.2461,1.0000,0.0000,0.5000,PO
3,-44,-44,-45,-45,760540,0.0816,0.0000,0.3306,0.0008,0.2480,0.9939,0.0052,0.5022,PO
4,-36,-36,2701616,2701616,763272,0.0997,0.0065,0.4187,0.1286,0.2540,0.5008,0.2686,0.5190,FS


In [9]:
df_id1 = df_king[["FID1", "ID1"]]
df_id2 = df_king[["FID2", "ID2"]]

In [15]:
df_id1 = df_id1.rename(columns={"FID1":"FID", "ID1" :"IID"})
df_id2 = df_id2.rename(columns={"FID2":"FID", "ID2" :"IID"})

df_id1.head()

,FID,IID
0,-1,-1
1,-11,-11
2,-12,-12
3,-44,-44
4,-36,-36


In [14]:
df_id2.head()

,FID,IID
0,4696286,4696286
1,4734664,4734664
2,4734664,4734664
3,-45,-45
4,2701616,2701616


In [19]:
len(df_id1) + len(df_id2)

58516

In [26]:
df_fid = pd.concat([df_id1, df_id2])
df_fid

,FID,IID
0,-1,-1
1,-11,-11
2,-12,-12
3,-44,-44
4,-36,-36
...,...,...
29253,5988287,5988287
29254,6024386,6024386
29255,5980583,5980583
29256,6019965,6019965


In [27]:
# any duplicates
assert not df_fid.duplicated().any(), "DataFrame contains duplicate rows"

AssertionError: DataFrame contains duplicate rows

In [28]:
# drop duplicates
df_fid_no_dup = df_fid.drop_duplicates()
df_fid_no_dup

,FID,IID
0,-1,-1
1,-11,-11
2,-12,-12
3,-44,-44
4,-36,-36
...,...,...
29251,5971936,5971936
29252,5988678,5988678
29253,5988287,5988287
29255,5980583,5980583


In [32]:
%%bash
dx download "project-J1VzYJQJQp6z67g2f0Vvb2kK:/user_data__Mahdi/i70_g6_CSF/a_man/all_ukb_IID_sex_ageAtRec.csv"
ls

all_ukb_IID_sex_ageAtRec.csv
array_data__kinship_degree1.kin0


In [33]:
df_agesex = pd.read_csv("all_ukb_IID_sex_ageAtRec.csv")
df_agesex

,eid,p31,p21022
0,1830117,Female,65
1,4103863,Male,52
2,1340728,Male,63
3,1357551,Male,48
4,1494901,Female,63
...,...,...,...
501956,4215286,Female,54
501957,3319318,Male,41
501958,2765625,Male,64
501959,2748139,Female,45


In [34]:
df_agesex = df_agesex.rename(columns = {"eid" : "IID", "p31":"sex", "p21022":"age"})

In [35]:
assert not df_agesex.duplicated().any(), "DataFrame contains duplicate rows"

In [36]:
df_agesex = df_agesex.merge(df_fid_no_dup, how='inner')

In [37]:
df_agesex

,IID,sex,age,FID
0,3514264,Female,67,3514264
1,4289082,Male,42,4289082
2,4906571,Female,67,4906571
3,3282010,Female,66,3282010
4,3059475,Female,47,3059475
...,...,...,...,...
52214,2728180,Male,44,2728180
52215,3754017,Male,64,3754017
52216,5197610,Female,63,5197610
52217,1037143,Female,61,1037143


In [38]:
# rearrange columns
df_agesex = df_agesex[["FID", "IID", "sex", "age"]]
df_agesex

,FID,IID,sex,age
0,3514264,3514264,Female,67
1,4289082,4289082,Male,42
2,4906571,4906571,Female,67
3,3282010,3282010,Female,66
4,3059475,3059475,Female,47
...,...,...,...,...
52214,2728180,2728180,Male,44
52215,3754017,3754017,Male,64
52216,5197610,5197610,Female,63
52217,1037143,1037143,Female,61


In [48]:
df_agesex.to_csv("agesex.txt", sep=' ', index = False)

In [49]:
%%bash
dx upload agesex.txt

ID                                file-J1xXq4jJQp6bG9xq0Pp7P3xV
Class                             file
Project                           project-J1VzYJQJQp6z67g2f0Vvb2kK
Folder                            /
Name                              agesex.txt
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Sat Jul 19 05:30:59 2025
Created by                        mahdimir
 via the job                      job-J1xKGZ8JQp6QzxyvkqyZK39v
Last modified                     Sat Jul 19 05:31:00 2025
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"


In [50]:
df_king

,FID1,ID1,FID2,ID2,N_SNP,HetHet,IBS0,HetConc,HomIBS0,Kinship,IBD1Seg,IBD2Seg,PropIBD,InfType
0,-1,-1,4696286,4696286,755432,0.0965,0.0064,0.3985,0.1260,0.2464,0.5543,0.2121,0.4893,FS
1,-11,-11,4734664,4734664,725130,0.0837,0.0000,0.3301,0.0005,0.2461,0.9995,0.0000,0.4997,PO
2,-12,-12,4734664,4734664,724414,0.0829,0.0000,0.3280,0.0006,0.2461,1.0000,0.0000,0.5000,PO
3,-44,-44,-45,-45,760540,0.0816,0.0000,0.3306,0.0008,0.2480,0.9939,0.0052,0.5022,PO
4,-36,-36,2701616,2701616,763272,0.0997,0.0065,0.4187,0.1286,0.2540,0.5008,0.2686,0.5190,FS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29253,5969734,5969734,5988287,5988287,764444,0.0838,0.0001,0.3293,0.0015,0.2453,0.9985,0.0000,0.4992,PO
29254,5973154,5973154,6024386,6024386,763054,0.0939,0.0082,0.3822,0.1573,0.2270,0.4852,0.2130,0.4556,FS
29255,5979383,5979383,5980583,5980583,760441,0.1008,0.0071,0.4210,0.1436,0.2521,0.4772,0.2587,0.4973,FS
29256,5982660,5982660,6019965,6019965,756991,0.1038,0.0057,0.4456,0.1161,0.2742,0.4664,0.3198,0.5530,FS


In [ ]:
# how many po pairs we have
len(df_king[df_king["InfType"].eq("PO")]